In [243]:
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end

"/Users/allisonlettiere/Desktop/CS238FinalProject/Project.toml"

In [244]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using StaticArrays
import Distributions.pdf

In [245]:
sensor = Bumper()
config = 3# 1,2, or 3

num_x_pts = 20
num_y_pts = 20
num_th_pts = 5
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
aspace = vec(collect(RoombaAct(v, om) for v in [0,1,10], om in [-1,0,1]))

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace))

RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  sspace: DiscreteRoombaStateSpace
  room: AA228FinalProject.Room
  aspace: Array{RoombaAct}((9,))
  _amap: Dict{RoombaAct,Int64}
)

In [246]:
num_particles = 1000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

mdp(e::RoombaMDP) = e
mdp(e::RoombaPOMDP) = e.mdp

mdp (generic function with 2 methods)

## Randomized Point Based Value Iteration Policy Building

In [247]:
discountFactor = 0.9
goal_xy = get_goal_xy(m)
goal_x, goal_y = goal_xy

RW = 5
if config == 1
    x = 10+RW 
    y = -RW
elseif config== 2
    x = -20+RW
    y = RW
else
    x = -20+RW
    y = RW
end

ang_to_goal = atan(goal_y - y, goal_x - x)
si = RoombaState(x,y,ang_to_goal,0.)
sf = RoombaState(goal_x,goal_y, 1, 0.)

if(ang_to_goal <= 0.5 && ang_to_goal >= (2*pi - 1) + 0.5)
    ang_to_goal = 0.0
elif(ang_to_goal > 0.5 && ang_to_goal <= (2*pi - 1) - 2.14)
    ang_to_goal = 1.0
else
    ang_to_goal = -1.0
end

action = (10, ang_to_goal)
startingReward = reward(m, si, action, sf)
firstAlphaVector = []

for state in 1:length(POMDPs.states(m))
    push!(firstAlphaVector, (1/(1-discountFactor)) * startingReward)
end

In [267]:
gamma = Dict()
gamma[action] = firstAlphaVector

B = []
dist = initialstate_distribution(m)
b = initialize_belief(belief_updater, dist)

for i in 1:1
    push!(B, b)
    b = resample(resampler, b, belief_updater.spf.rng)
end

gamma = RandomizedPointBasedUpdate(B, gamma)

UndefVarError: UndefVarError: s not defined

In [249]:
function probabilityVector(m, b)    
    probabilitiesB = []
    ##if the state is not in the belief space, give probability 0
    for i in POMDPs.states(m)
        if(i in keys(probdict(b)))
            push!(probabilitiesB, probdict(b)[i])
        else
            push!(probabilitiesB, 0.0)
        end
    end
    return probabilitiesB
end

probabilityVector (generic function with 1 method)

In [238]:
function probdict(b::AbstractParticleBelief{S}) where {S}
    probs = Dict{S, Float64}()
    for (i,p) in enumerate(particles(b))
        if haskey(probs, p)
            probs[p] += weight(b, i)/weight_sum(b)
        else
            probs[p] = weight(b, i)/weight_sum(b)
        end
    end
    return probs
end

probdict (generic function with 1 method)

In [237]:
function RandomizedPointBasedUpdate(B, gamma)
    BP = B
    gammaP = Dict()
    #println(BP)
    while !(isempty(BP))
        b = rand(BP)
        filter!(e->e != b,BP)
        alphaVectors = backupBelief(b, gamma)
        for action in aspace
            if (action in gammaP)
                maxU = 0
                oldMaxAlpha = gammaP[action]
                oldUtility = oldMaxAlpha'probabilityVector(m, b)
                newUtility = alphaVectors[action]'probabilityVector(m, b)
                if (newUtility >= oldUtility)
                    gammaP[action] = alphaVectors[action]
                else
                    gammaP[action] = oldMaxAlpha
                end
            else
                gammaP[action] = alphaVectors[action]
            end
        end  
    end
    return gammaP
end

RandomizedPointBasedUpdate (generic function with 1 method)

In [266]:
function backupBelief(b, gamma)
    O = [0, 1]
    alphaAO = Dict()
    alphaVectors = Dict()
    actionNumber = 1
    for action in aspace
        for observation in O
            bp = update(belief_updater, b, action, observation)
            maxTranspose = 0
            for a in keys(gamma)
                t = gamma[a]'probabilityVector(m, bp)
                if (t > maxTranspose)
                    maxTranspose = t
                    alphaAO[[action, observation + 1]] = gamma[a]
                end
            end
        end 
        
        s = mean(b)
        e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status = collisionStatus(s, action)
        sp = RoombaState(next_x, next_y, next_th, next_status)

        alpha = Dict()
        
        stateNumber = 1
        for state in POMDPs.states(m)          
            alpha[[stateNumber, actionNumber]] = reward(m, s, action, sp) + discountFactor * sumNextStateObservation(s, action, alphaAO, stateNumber)
            stateNumber += 1
        end
        alphaVectors[action] = alpha[[:,actionNumber]]
        actionNumber += 1
    end
    return alphaVectors
end     

backupBelief (generic function with 1 method)

In [234]:
function sumNextStateObservation(state, action, alphaAO, stateNumber)
    sum = 0
    e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp = collisionStatus(state, action)
    
    ##0 -- does not hit wall (false), 1 -- hits wall (true)
    for o in [0, 1]
        observationFunction = 0
        if o == 0
            if(!wall_contact(e, s))
                observationFunction = 1
                ob = 0
            end
        elseif o == 1
            if(wall_contact(e, s))
                observationFunction = 1
            end
        end
        transitionFunction = 1 
        sum += observationFunction * transitionFunction * alphaAO[[action, o + 1]][stateNumber]
    end
    return sum
end

sumNextStateObservation (generic function with 1 method)

In [233]:
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    maxUtility = 0
    actionSelection = 0
    for action in gamma
        alpha = gamma[action]
        transpose = alpha'probabilityVector(m, b)
        if (transpose > maxUtility)
            maxUtility = transpose
            actionSelection = action
        end
    end
    return RoombaAct(actionSelection[1], actionSelection[2])
end

## Create and Run the Environment

In [35]:
function runEnvironment()
    # first seed the environment
    Random.seed!(2)

    # reset the policy
    p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

    # run the simulation
    c = @GtkCanvas()
    win = GtkWindow(c, "Roomba Environment", 600, 600)
    for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
        @guarded draw(c) do widget

            # the following lines render the room, the particles, and the roomba
            ctx = getgc(c)
            set_source_rgb(ctx,1,1,1)
            paint(ctx)
            render(ctx, m, step)

            # render some information that can help with debugging
            # here, we render the time-step, the state, and the observation
            move_to(ctx,300,400)
            show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
        end
        show(c)
        sleep(0.1) # to slow down the simulation
    end
end

runEnvironment (generic function with 1 method)

In [81]:
@time runEnvironment()

BoundsError: BoundsError

### Helper Functions

In [116]:
function reward(m::RoombaPOMDP{Bumper,Bool}, s::RoombaState, a::Tuple{Int64,Float64}, sp::RoombaState)
    
    # penalty for each timestep elapsed
    cum_reward = mdp(m).time_pen

    # penalty for bumping into wall (not incurred for consecutive contacts)
    previous_wall_contact = wall_contact(m,s)
    current_wall_contact = wall_contact(m,sp)
    if(!previous_wall_contact && current_wall_contact)
        cum_reward += mdp(m).contact_pen
    end

    # terminal rewards
    cum_reward += mdp(m).goal_reward*(sp.status == 1.0)
    cum_reward += mdp(m).stairs_penalty*(sp.status == -1.0)

    return cum_reward  
end

reward (generic function with 2 methods)

In [117]:
function update(up::RoombaParticleFilter, b::ParticleCollection{RoombaState}, a, o)
    ps = particles(b)
    pm = up.spf._particle_memory
    wm = up.spf._weight_memory
    resize!(pm, 0)
    resize!(wm, 0)
    sizehint!(pm, n_particles(b))
    sizehint!(wm, n_particles(b))
    all_terminal = true
    for i in 1:n_particles(b)
        s = ps[i]
        if !isterminal(up.spf.model, s)
            all_terminal = false
            # noise added here:
            a_pert = a + SVector(up.v_noise_coeff*(rand(up.spf.rng)-0.5), up.om_noise_coeff*(rand(up.spf.rng)-0.5))
            sp = generate_s(up.spf.model, s, a_pert, up.spf.rng)
            push!(pm, sp)
            push!(wm, obs_weight(up.spf.model, s, a_pert, sp, o))
        end
    end
    # if all particles are terminal, return previous belief state
    if all_terminal
        return b
    end

    return resample(up.spf.resample, WeightedParticleBelief{RoombaState}(pm, wm, sum(wm), nothing), up.spf.rng)
end

update (generic function with 2 methods)

In [262]:
function collisionStatus(s, a)
    e = mdp(m)
    v, om = action
    v = clamp(v, 0.0, e.v_max)
    om = clamp(om, -e.om_max, e.om_max)

    # propagate dynamics without wall considerations
    x, y, th, _ = s
    dt = e.dt

    # dynamics assume robot rotates and then translates
    next_th = wrap_to_pi(th + om*dt)

    # make sure we arent going through a wall
    p0 = SVector(x, y)
    heading = SVector(cos(next_th), sin(next_th))
    des_step = v*dt
    next_x, next_y = AA228FinalProject.legal_translate(e.room, p0, heading, des_step)

    # Determine whether goal state or stairs have been reached
    grn = mdp(m).room.goal_rect
    gwn = mdp(m).room.goal_wall
    srn = mdp(m).room.stair_rect
    swn = mdp(m).room.stair_wall
    gr = mdp(m).room.rectangles[grn]
    sr = mdp(m).room.rectangles[srn]    
    
    next_status = 1.0*AA228FinalProject.contact_wall(gr, gwn, [next_x, next_y]) - 1.0*AA228FinalProject.contact_wall(sr, swn, [next_x, next_y])
    sp = RoombaState(next_x, next_y, next_th, next_status)
    
    return e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp
end

collisionStatus (generic function with 1 method)